### For part 1, question 2

In [1]:
import numpy as np
from cvxopt import matrix, solvers

# Data points
X = np.array([[3, 3], [4, 3], [1, 1]])
y = np.array([1, 1, -1])

# Number of data points
n = X.shape[0]

# Compute the Gram matrix (K)
K = np.dot(X, X.T) * np.outer(y, y)

# Define the quadratic programming problem
P = matrix(K, tc='d')
q = matrix(-np.ones(n), tc='d')

# Non-negativity constraints (alpha_i >= 0)
G = matrix(-np.eye(n), tc='d')
h = matrix(np.zeros(n), tc='d')

# Equality constraints (sum(alpha_i * y_i) = 0)
A = matrix(y, (1, n), tc='d')
b = matrix(0.0, tc='d')

# Solve the quadratic programming problem
sol = solvers.qp(P, q, G, h, A, b)

# Extract the solution (alpha values)
alphas = np.array(sol['x']).flatten()

print("Optimal alpha values:", alphas)

     pcost       dcost       gap    pres   dres
 0: -2.6873e-01 -6.7301e-01  4e+00  2e+00  1e+00
 1: -1.4232e-01 -5.9278e-01  5e-01  3e-16  6e-16
 2: -2.3773e-01 -2.7513e-01  4e-02  6e-17  5e-16
 3: -2.4959e-01 -2.5037e-01  8e-04  6e-17  4e-16
 4: -2.5000e-01 -2.5000e-01  8e-06  6e-17  2e-16
 5: -2.5000e-01 -2.5000e-01  8e-08  3e-17  3e-16
Optimal solution found.
Optimal alpha values: [2.49999916e-01 8.17639192e-08 2.49999998e-01]
